# Interface for publication on Invenio with widgets

I want to provide a graphical interface for publishing GMAP packages to our Invenio instance.
We have a working version of a client API (in Python), but for those not really into Python, it would be nice to provide something simple, high-level.

We should start from GMAP metadata model, published in the wiki:
- https://wiki.europlanet-gmap.eu/bin/view/Main/Documentation/Map-wide%20metadata/

The fields and meaning of GMAP packages metadata set are:

| Field | Field description | Comments |
| --- | --- | --- |
| _Map name (GMAP_ID)_ | Unique package name | Format: "`GMAP-{{target-body}}-{{content-type}}-{{region-label}}_{{detail-label}}`". See **Note-1**. |
| Target body | Name of target body | _eg_, `Mercury`, `Moon` |
| Title of map | Map title | _eg_, `Awesome Geologic Map of the region X` |
| Bounding box - Min Lat | Minimum latitude in degrees [-90:90) (< Max Lat) | |
| Bounding box - Max Lat | Maximum latitude in degrees (-90:90] (> Min Lat) | |
| Bounding box - Min Lon | West-most Longitude in degrees [-180:180) (< Max Lon) | |
| Bounding box - Max Lon | East-most Longitude in degrees (-180:180] (> Min Lon) | |
| Author(s) | Semi-colon separated list of authors | 
| Type | Either "draft" or "released" | See **Note-2** |
| Output scale | Map spatial scale |
| Original Coordinate Reference System | WKT declaring map' CRS |
| Data used | Semi-colon separated list of ancillary, original data used |
| Standards adhered to | Semi-colon list of standards used in the map |
| DOI of companion paper(s) | DOI of linked publication |
| Aims | Reason, goal for this map |
| Short description | Free-text (500 words maximum) describing the map |
| Related products | Other geological maps complementing this one |
| Units Definition (polygon styling) | Units color definition |
| Stratigraphic info | Description of stratigraphic elements in the map |
| Other comments | free-text (notes, errata, warnings) |
| Heritage used | heritage information |
| Link to other data | Links to extenal resources |
| Acknowledgements | Free-text acknowledge |

> **Note-1**: in GMAP's progenitor project -- PLANMAP --,  "Map name" was given by the data package author after following a set of rules to fill the ID template (_ie_, _target-body_, _content-type_ and _labels_). This manual work does not scale very well; it did work just fine during PLANMAP, but if we want to guarantee a standard format the best thing to do is (1) collect those values separately (_eg_, "content-type"), and then (2) compose them in a _ID_ string ourselves; that allows for ID validation.
>
> That being said, we have to modify the initial (GMAP) metadata model above to:
> 1. include "content-type" and "region-/detail-label" fields;
> 2. turn "map-name (gmap-id)" into a read-only field.
>
> GMAP naming conventions (_ie_, GMAP-ID designation) is explained at:
> - https://wiki.europlanet-gmap.eu/bin/view/Main/Documentation/GMAP%20naming%20conventions/

> **Note-2**: the `Type` keyword is ambiguous, and using it to communicate the production stage the package is currently in ("draft" or "released"). A more meaningful keyword would be "`Status`"


In [1]:
import ipywidgets as widgets

from IPython.display import display

In [10]:
# Map title
#
title = widgets.Text(
    description = 'Map title'
)

title

Text(value='', description='Map title')

In [11]:
# Target body: a pre-defined list of planets and satelites
# - mandatory
target = widgets.Combobox(
    description = 'Target body',
    placeholder = 'Choose a body',
    options = ['Mars', 'Mercury', 'Moon'],
    ensure_option = True,
)

target

Combobox(value='', description='Target body', ensure_option=True, options=('Mars', 'Mercury', 'Moon'), placeho…

In [12]:
_map_types = {
    'Integrated': 'I',
    'Morphologic': 'M',
    'Stratigraphic': 'S',
    'Compositional': 'C',
    'Digital model': 'D',
    'Geo-structural': 'G',
}

map_type = widgets.SelectMultiple(
    description = 'Map type',
    options = sorted(_map_types.keys()),
)

map_type

SelectMultiple(description='Map type', options=('Compositional', 'Digital model', 'Geo-structural', 'Integrate…

In [13]:
label = widgets.Text(
    description = 'Map label',
    placeholder = 'Format: "Label", "Label-sublabel"'
)

label

Text(value='', description='Map label', placeholder='Format: "Label", "Label-sublabel"')

In [14]:
gmap_id = widgets.Text(
    description = 'GMAP-ID',
    disabled = True
)

def on_change_value_for_id(change):
    _target = target.value.strip()
    _type = ''.join([ _map_types[t] for t in map_type.value ])
    _label = label.value.strip().replace(' ','-').title()
    _id = ['GMAP', _target, _type, _label]
    if not all(_id):
        gmap_id.value = "not-valid"
    else:
        _id = '_'.join(_id)
        gmap_id.value = _id
        
target.observe(on_change_value_for_id, names='value')
map_type.observe(on_change_value_for_id, names='value')
label.observe(on_change_value_for_id, names='value')

display(gmap_id)

Text(value='', description='GMAP-ID', disabled=True)

In [15]:
import datetime

date = widgets.DatePicker(
    description = 'Publication date',
    value = datetime.date.today()
)

date

DatePicker(value=datetime.date(2022, 10, 17), description='Publication date:', step=1)

In [18]:
widgets.VBox(
    [title,
     target,
     map_type,
     label,
     gmap_id,
     date]
)